<a href="https://colab.research.google.com/github/Sourasky-DHLAB/Whisper/blob/main/Whisper_Audio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div style="direction:rtl" dir="rtl">
    <img src="https://drive.google.com/uc?export=view&amp;id=15xQaGVrJej6UIPjfAB-QVWXLWVChPJSt"><br>
<div style="direction:rtl" dir="rtl">
  <p style="text-align: right; direction: rtl; float: right;">    נכתב על ידי <a href="mailto:odedzarchia@tauex.tau.ac.il">עודד זרחיה</a> | <a href="https://github.com/Sourasky-DHLAB">Github</a><br>
    <a href="https://cenlib.tau.ac.il/">הספרייה המרכזית ע"ש סוראסקי</a> | <a href="https://tau.ac.il/">אוניברסיטת תל אביב</a>
</div></p>

<div dir="rtl">
<h1><strong>1. תמלול קבצי אודיו באמצעות Whisper</strong></h1>
<p>
<div style="direction:rtl" dir="rtl">
  <p style="text-align: right; direction: rtl; float: right;">
Whisper הוא מודל לזיהוי דיבור מבית <a href="https://openai.com/">OpenAI</a> הזמין לציבור הרחב בקוד פתוח. מודל זה אומן על יותר מ-680 אלף שעות של שיחות באנגלית ובשפות רבות אחרות – בהן גם עברית וערבית.<br>
מחברת זו תדריך אתכם בתמלול קטעי אודיו באמצעות Whisper. תוכלו להשתמש במחברת כפי שהיא כדי לאחסן את קבצי התמליל ב-Google Drive.<br>
לתשומת לבכם - השימוש במחברת זו ובמודל של Whisper חופשי לחלוטין וללא שום עלות. בנוסף, אין הגבלה על אורך קטעי הוידאו/אודיו שניתן לתמלל באמצעות Whisper.
</p>
</div>

<h1><strong><div style="direction:rtl" dir="rtl">2. בדוק את סוג המעבד הגרפי (GPU) </div></strong></h1>
<div style="direction:rtl" dir="rtl">
  <p style="text-align: right; direction: rtl; float: right;">סוג <a href="https://https://he.wikipedia.org/wiki/%D7%9E%D7%A2%D7%91%D7%93_%D7%92%D7%A8%D7%A4%D7%99">(המעבד הגרפי)</a> <strong>(GPU - Graphics Processing Unit)</strong> שאתם מקבלים ב-Google Colab מגדיר את המהירות שבה קטעי האודיו יתומללו. ככל שמספר <a href="https://https://he.wikipedia.org/wiki/FLOPS">(הפלופס)</a> <strong>(FLOPS - Floating Point Operations Per Second, פעולות נקודות צפות לשנייה)</strong> גבוה יותר, כך התמלול מהיר יותר. יחד עם זאת, גם המעבד הגרפי החלש ביותר ב-Colab מסוגל להריץ כל מודל של Whisper. יש לוודא כי בחרתם ב-GPU כמאיץ חומרה עבור מחברת זו (Runtime → Change runtime type → Hardware accelerator).</p>
</div>

<table>
  <thead>
    <tr>
      <th>GPU</th>
      <th>GPU RAM</th>
      <th>FP32 teraFLOPS</th>
      <th>Availability</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>T4</td>
      <td>16 GB</td>
      <td>8.1</td>
      <td>Free</td>
    </tr>
    <tr>
      <td>P100</td>
      <td>16 GB</td>
      <td>10.6</td>
      <td>Colab Pro</td>
    </tr>
    <tr>
      <td>V100</td>
      <td>16 GB</td>
      <td>15.7</td>
      <td>Colab Pro (Rare)</td>
    </tr>
  </tbody>
</table>


<div style="direction:rtl" dir="rtl">
  <p style="text-align: right; direction: rtl; float: right;">
הריצו את התא הבא כדי לוודא את סוג המעבד הגרפי שהוקצה עבור מחברת זו. אפסו את ה-runtime של המחברת אם ברצונכם לקבל מעבד גרפי אחר (Runtime → Restart runtime).</p>

In [ ]:
# check gpu type
!nvidia-smi -L
!nvidia-smi

<div style="direction:rtl" dir="rtl"><h1><strong>3. התקנת ספריות</strong></h1>
<div style="direction:rtl" dir="rtl">
  <p style="text-align: right; direction: rtl; float: right;">הריצו את התא הבא כדי להוריד ולהתקין את ספריית <a href="https://github.com/openai/whisper">Whisper</a> הנחוצה לפעולת התמלול.</p></div>

In [ ]:
# install libraries
!pip install git+https://github.com/openai/whisper.git 
!pip install jiwer

<div style="direction:rtl" dir="rtl">
  <h1><strong>4. ייבוא ספריות </strong></h1>
  <div style="direction:rtl" dir="rtl">
  <p style="text-align: right; direction: rtl; float: right;">הריצו את התא הבא כדי לייבא את הספריות הנדרשות עבור פעולת התמלול.</p>
</div>

In [63]:
import whisper
from whisper.utils import get_writer
import os
import string
from jiwer import wer

<h1><strong><div style="direction:rtl" dir="rtl">5. חיבור ל-Google Drive </div></strong></h1>
<div style="direction:rtl" dir="rtl">
  <p style="text-align: right; direction: rtl; float: right;">הריצו את התא הבא כדי להתחבר ל-Google Drive האישי שלכם. כדי לראות את הקבצים שלכם פתחו את סייר הקבצים בתפריט השמאלי.</p></div>

In [ ]:
# Load all the audio file paths in a Google Drive folder
from google.colab import drive
drive.mount("/content/drive", force_remount=True) # This will prompt for authorization.

<h1 dir="rtl"><strong>6. הגדרת תיקיות </strong></h1>
<div style="direction:rtl" dir="rtl">
<div style="direction:rtl" dir="rtl">
  <p style="text-align: right; direction: rtl; float: right;">הריצו את התא הבא כדי להגדיר את מיקום קבצי האודיו והתמליל:
<ol>
<li>קבצי אודיו - מחברת זו מניחה כי קבצי האודיו לתמלול נמצאים ב-Google Drive תחת תיקיית <code>Whisper/Audio</code>. ניתן לשנות הגדרה זו במשתנה <code>audio_folder</code>.</li>
<li>קבצי תמליל - מחברת זו מניחה כי קבצי הטקסט של התמלול יישמרו ב-Google Drive תחת תיקיית <code>Whisper/Transcriptions</code>. ניתן לשנות הגדרה זו במשתנה <code>transcription_folder</code>. במידה ואינה קיימת, תיקייה זו תיווצר באופן אוטומטי לאחר הרצת התא.</li>
</ol>
</p></div>

In [5]:
# Assuming the audio files are in a folder called "Audio" under "Whisper"
audio_folder = "/content/drive/MyDrive/Whisper/Audio/"

# Assuming the text files will be placed in a folder called "Transcriptions" under "Whisper"
transcription_folder = "/content/drive/MyDrive/Whisper/Transcriptions/"

# Create "Transcriptions" folder if does not exist
if not os.path.exists(transcription_folder):
  os.makedirs(transcription_folder)

<h1><strong><div style="direction:rtl" dir="rtl">7. טעינת קבצי אודיו לתמלול </div></h1></strong>
<div style="direction:rtl" dir="rtl">
<p>הריצו את התא הבא כדי לטעון את קבצי האודיו הנמצאים בתיקיית <code>Whisper/Audio</code> לתוך משתנה מסוג <a href="https://he.wikipedia.org/wiki/%D7%A8%D7%A9%D7%99%D7%9E%D7%94_(%D7%9E%D7%91%D7%A0%D7%94_%D7%A0%D7%AA%D7%95%D7%A0%D7%99%D7%9D)">רשימה</a> (list). הפלט יציג את שמות וכמות הקבצים לתמלול.</p>

In [ ]:
# Get a list of all the file paths in the folder
audio_files = []
for file in os.listdir(audio_folder):
    audio_files.append(audio_folder + file)
for p in audio_files:
    print(p)
print(f"\033[1m There are {len(audio_files)} audio files to transcribe")

<h1 id="-div-style-direction-rtl-dir-rtl-8-div-"><strong><div style="direction:rtl" dir="rtl">8. טעינת מודל התמלול </div></strong></h1>
<div style="direction:rtl" dir="rtl">

הריצו את התא הבא כדי לטעון את מודל התמלול המתאים.
יושם לב כי אנו משתמשים במודל הבינוני (<strong>medium</strong>), וזאת משום שפעולת התמלול צורכת משאבי מחשוב רבים ועלולה לְהַקְרִיס את המחברת. <br>
ישנם חמישה מודלים, ארבעה עם גרסאות באנגלית בלבד, המציעים פשרות שונות בין מהירות לדיוק. בטבלה להלן מוצגים שמות המודלים הזמינים יחד עם דרישות הזיכרון והמהירות היחסית שלהם:
</div>

<table>
<thead>
<tr>
<th style="text-align:center">Size</th>
<th style="text-align:center">Parameters</th>
<th style="text-align:center">English-only model</th>
<th style="text-align:center">Multilingual model</th>
<th style="text-align:center">Required VRAM</th>
<th style="text-align:center">Relative speed</th>
</tr>
</thead>
<tbody>
<tr>
<td style="text-align:center">tiny</td>
<td style="text-align:center">39 M</td>
<td style="text-align:center"><code>tiny.en</code></td>
<td style="text-align:center"><code>tiny</code></td>
<td style="text-align:center">~1 GB</td>
<td style="text-align:center">~32x</td>
</tr>
<tr>
<td style="text-align:center">base</td>
<td style="text-align:center">74 M</td>
<td style="text-align:center"><code>base.en</code></td>
<td style="text-align:center"><code>base</code></td>
<td style="text-align:center">~1 GB</td>
<td style="text-align:center">~16x</td>
</tr>
<tr>
<td style="text-align:center">small</td>
<td style="text-align:center">244 M</td>
<td style="text-align:center"><code>small.en</code></td>
<td style="text-align:center"><code>small</code></td>
<td style="text-align:center">~2 GB</td>
<td style="text-align:center">~6x</td>
</tr>
<tr>
<td style="text-align:center">medium</td>
<td style="text-align:center">769 M</td>
<td style="text-align:center"><code>medium.en</code></td>
<td style="text-align:center"><code>medium</code></td>
<td style="text-align:center">~5 GB</td>
<td style="text-align:center">~2x</td>
</tr>
<tr>
<td style="text-align:center">large</td>
<td style="text-align:center">2870 M</td>
<td style="text-align:center">N/A</td>
<td style="text-align:center"><code>large</code></td>
<td style="text-align:center">~10 GB</td>
<td style="text-align:center">1x</td>
</tr>
</tbody>
</table>
<p><div style="direction:rtl" dir="rtl"><br></p>
<p>במידה וברצונכם להתשמש במודל המדויק ביותר לעברית (<strong>large</strong>) עבור קבצים מרובים, מומלץ להריץ את המחברת בסביבה עם משאבי מחשוב מובטחים כדוגמת Google Colab Pro. עבור תמלול באנגלית בלבד, מודלי ה-en. נוטים לתפקד טוב יותר.</p>

In [ ]:
# load whisper model
model = whisper.load_model("tiny")
print(f"\033[1m Model loaded successfully")

<h1 id="-div-style-direction-rtl-dir-rtl-9-div-"><strong><div style="direction:rtl" dir="rtl">9. קביעת שפה לתמלול וסוג הפלט</div></strong></h1>
<div style="direction:rtl" dir="rtl">

<p>הריצו את התא הבא כדי לקבוע שני משתנים:</p>
<ol>
<li><code>lang</code> - הגדרת שפת התמלול. <code>he</code> משמעו עברית. ניתן להדפיס את רשימת השפות הנתמכת על ידי הרצת הפקודה <code>print(whisper.tokenizer.LANGUAGES)</code> בתא נפרד.<br></li>
<li><p><code>output_format</code> - הגדרת סוג הפלט. Whisper תומך בהפקת הפלטים הבאים:</p>
<ul>
<li><code>txt</code> - קובץ טקסט עם מעברי שורות, ניתן לפתיחה בכל מעבד תמלילים.</li>
<li><code>srt/vtt</code>- קבצי כתוביות המכילים טקסט וחתימות זמן.</li>
<li><code>tsv</code>- קובץ טקסט מופרד בטאבים עם חלוקה לסגמנטים. ניתן לפתיחה כגיליון אלקטרוני.</li>
<li><code>json</code>- מבנה מידע המורכב מזוגות של מפתח-ערך.<br></li>
</ul>
<p>החליפו את ערך המשתנה בהתאם לסוג הפלט שברצונכם לקבל. ניתן להפיק את כל סוגי הפלטים באמצעות קביעת הערך <code>all</code>. </p>
</li>
</ol>

In [138]:
lang = 'he'
output_format = 'txt'

<h1 id="-div-style-direction-rtl-dir-rtl-10-div-"><strong><div style="direction:rtl" dir="rtl">10. תמלול </div></strong></h1>
<div style="direction:rtl" dir="rtl">

<p>הריצו את התא הבא כדי לתמלל את קבצי האודיו בתיקיית <code>Whisper/Audio</code>. 
קבצי הטקסט יישמרו בתיקיית <code>Whisper/Transcriptions</code> עם שם זהה לקבצי האודיו אך עם סיומת txt. </p>

In [ ]:
# set timer
import time
start_time = time.time()
# transcribe audio files in list
for p in audio_files:    
  result = model.transcribe(p, verbose = False, language = lang) # to translate add task = 'transalte'
  # use get_writer method to output files
  output_file = get_writer(output_format, transcription_folder)
  output_file(result, p[:-4])
  print(p)
print(f"\033[1m--- Transcribed {len(audio_files)} audio files in %s seconds ---" % (time.time() - start_time))

<h1 id="-div-style-direction-rtl-dir-rtl-11-div-"><strong><div style="direction:rtl" dir="rtl">11. בדיקת איכות (אופציונלי) </div></strong></h1>
<div style="direction:rtl" dir="rtl">

<p>הרצת התאים הבאים תאפשר לכם להעריך את הדיוק של התמלול שהפקתם באמצעות Whisper. בדיקה זו מתבצעת במספר שלבים, ומצריכה את קובץ התמלול המקורי (<a href="https://https://en.wikipedia.org/wiki/Ground_truth">Ground Truth</a>): </p>
<ol>
<li>נירמול פלט התמלול של Whisper וקובץ התמלול המקורי  - הסרת סימני פיסוק, רווחים מיותרים, קפיטליזציה וכו&#39;. הנתיב של הקובץ שהפקתם באמצעות Whisper מוגדר במשתנה מוגדר במשתנה <code>whisper_output</code>. הנתיב של קובץ התמלול המקורי מוגדר במשתנה <code>ground_truth</code>.</li>
<li>השוואה בין שני הקבצים וחישוב אחוז השגיאות בהתאם למדד <a href="https://https://en.wikipedia.org/wiki/Word_error_rate">Word Error Rate</a> - באמצעות ספריית jiwer.</li>
</ol>
<p>הפלט המתקבל מוצג באחוזים ומצביע על אחוז המילים השגויות בקובץ התמלול, וזאת בהשוואה לקובץ המקור. כלומר, במידה ואחוז השגיאות עומד על כ-3%, אזי מתוך 100 מילים יש 3 מילים שגויות.</p>

In [ ]:
# normalize whisper output
whisper_output = '/content/drive/MyDrive/Whisper/Transcriptions/Copy of Snyder.txt'
whisper_output_norm = open(whisper_output, 'r').read().lower().translate(str.maketrans('', '', string.punctuation)).replace('\n', ' ')
print(whisper_output_norm)

In [ ]:
# normalize ground truth
ground_truth = '/content/drive/MyDrive/Whisper/GT/Snyder_GT.txt'
ground_truth_norm = open(ground_truth, 'r').read().lower().translate(str.maketrans('', '', string.punctuation)).replace('\n', ' ')
print(ground_truth_norm)

In [ ]:
# calculate WER
reference = ground_truth_norm
hypothesis = whisper_output_norm
error = wer(reference, hypothesis)
error_percentage = "{:.0%}".format(error)
print(error_percentage)